# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 10
importance_type = "split"

life_history = ["lifespan", "mass_kg", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

#lgb_params["importance_type"] = importance_type

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector
from yspecies.results import FeatureSummary

NameError: name 'Nonee' is not defined

In [ ]:
import optuna
from optuna import Study, Trial
from optuna import multi_objective
from optuna.multi_objective import trial
from optuna.multi_objective.study import MultiObjectiveStudy
from yspecies.tuning import MultiObjectiveResults

In [ ]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [5]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [6]:
from loguru import logger
logger.add(locations.logs / "stage_two_shap_selection.log", rotation="12:00")     # New file is created each day at noon

1

In [7]:
stage_2_folder = locations.interim.dir / "stage_2" 
stage_2_folder.mkdir(exist_ok=True)
partitions_folder = stage_2_folder  / "partitions"
partitions_folder.mkdir(exist_ok=True)

In [8]:
data = ExpressionDataset.from_folder(locations.interim.dir / "stage_2" / "input")
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 142)","(142, 37)",38,408,"(142, 2)","(38, 18)"


## Setting up features to select ##

In [9]:
default_selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
default_selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,[]


In [10]:
selections = OrderedDict([(trait, (data,  replace(default_selection, to_predict = trait))) for trait in life_history])
selections["lifespan"][0]

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(408, 142)","(142, 37)",38,408,"(142, 2)","(38, 18)"


### Setting up pipelines ###

In [19]:
from yspecies.helpers import PipelineFactory
factory = PipelineFactory(locations,repeats, n_folds, n_hold_out)

In [20]:
def opt_by_trait_2(trait: str):
    return locations.interim.optimization / f"{trait}_2.sqlite"

In [21]:
print("setting up quick pipelines")
quick_pipelines =  {trait:factory.make_shap_pipeline(study_path = opt_by_trait_2(trait), study_name=f"{trait}_r2_huber_kendall") for trait in life_history}
print("setting up pipelines with repeats")
pipelines = {trait:factory.make_repeated_shap_pipeline(study_path = opt_by_trait_2(trait), study_name=f"{trait}_r2_huber_kendall") for trait in life_history}
#pipelines = {trait:factory.make_shap_pipeline(trait) for trait in life_history}

#make_partitioning_shap_pipeline
pipelines.keys()

setting up quick pipelines
loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/lifespan_2.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-09-21 01:00:50,032] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/mass_kg_2.sqlite


[I 2020-09-21 01:00:51,190] Using an existing study with name 'mass_kg_r2_huber_kendall' instead of creating a new one.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/mtGC_2.sqlite


...

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/temperature_2.sqlite


[I 2020-09-21 01:00:52,501] Using an existing study with name 'temperature_r2_huber_kendall' instead of creating a new one.


loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/optimization/gestation_days_2.sqlite


[I 2020-09-21 01:00:52,523] Using an existing study with name 'gestation_days_r2_huber_kendall' instead of creating a new one.


dict_keys(['lifespan', 'mass_kg', 'mtGC', 'metabolic_rate', 'temperature', 'gestation_days'])

# Run second stage #

In [22]:
stage_two = {}

In [23]:
def run_stage_two(trait: str, quick: bool = False):
    pipes = quick_pipelines if quick else pipelines
    result = pipes[trait].fit_transform(selections[trait])
    stage_two[trait] = result #ugly mutation of stage_one dicionary
    return result

In [25]:
lifespan_2.selected.to_csv(locations.output.stage_two / "lifespan.tsv", sep="\t", index_label="reference_gene")

In [24]:
lifespan_2 = run_stage_two("lifespan", False)
lifespan_2.selected

2020-09-21 01:00:56.474 | INFO     | yspecies.selection:fit:149 - ===== fitting models with seed 0 =====
2020-09-21 01:00:56.475 | INFO     | yspecies.selection:fit:150 - PARAMETERS:
{'bagging_fraction': 0.757456996980591, 'boosting_type': 'gbdt', 'drop_rate': 0.15938150347995844, 'feature_fraction': 0.41345305892024875, 'lambda_l1': 2.9528782080363634, 'lambda_l2': 2.3536556238167003, 'learning_rate': 0.05373601286514567, 'max_depth': 3, 'max_leaves': 19, 'min_data_in_leaf': 8, 'objective': 'regression', 'metrics': ['l1', 'l2', 'huber'], 'verbose': -1, 'early_stopping_round': 10}
2020-09-21 01:00:56.479 | INFO     | yspecies.selection:fit:153 - SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Homo_sapiens', 'Equus_caballus']
Found `early_stopping_round` in params. Will use it instead of argument


Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[117]	valid_0's l1: 21.5888	valid_0's l2: 1953.78	valid_0's huber: 19.1044


2020-09-21 01:00:56.558 | INFO     | yspecies.selection:fit:153 - SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Ursus_americanus', 'Phascolarctos_cinereus']


Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
[250]	valid_0's l1: 3.26036	valid_0's l2: 38.7967	valid_0's huber: 2.62994
Did not meet early stopping. Best iteration is:
[250]	valid_0's l1: 3.26036	valid_0's l2: 38.7967	valid_0's huber: 2.62994


2020-09-21 01:00:56.670 | INFO     | yspecies.selection:fit:153 - SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Suricata_suricatta', 'Mus_spicilegus']


...

Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[175]	valid_0's l1: 2.48341	valid_0's l2: 14.9513	valid_0's huber: 1.88996


2020-09-21 01:02:51.831 | INFO     | yspecies.selection:fit:153 - SEED: 9 | FOLD: 3 | VALIDATION_SPECIES: ['Homo_sapiens', 'Otolemur_garnettii']


Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
[250]	valid_0's l1: 21.7979	valid_0's l2: 1941.3	valid_0's huber: 19.2941
Did not meet early stopping. Best iteration is:
[249]	valid_0's l1: 21.7976	valid_0's l2: 1941.42	valid_0's huber: 19.2943


2020-09-21 01:03:04.841 | INFO     | yspecies.explanations:transform:494 - Metrics: 
Metrics(R2=0.6538523640752132, MAE=8.306377456176657, MSE=518.0797102896187, huber=7.122683660075034)


,symbol,repeats,mean_shap,mean_kendall_tau,shap_0,kendall_tau_0,shap_1,kendall_tau_1,shap_2,kendall_tau_2,...,shap_5,kendall_tau_5,shap_6,kendall_tau_6,shap_7,kendall_tau_7,shap_8,kendall_tau_8,shap_9,kendall_tau_9
ENSG00000167515,TRAPPC2L,10.0,22.250,-0.286302,28.50,-0.321438,15.75,-0.222454,19.50,-0.262519,...,20.00,-0.198476,21.00,-0.216406,24.25,-0.467742,23.00,-0.259238,20.25,-0.318612
ENSG00000010219,DYRK4,10.0,21.300,0.499266,19.25,0.463278,32.25,0.442642,21.00,0.531262,...,26.25,0.465746,24.50,0.568590,17.75,0.508938,20.50,0.359008,21.75,0.503933
ENSG00000165501,LRR1,10.0,18.725,0.714162,18.50,0.718542,21.00,0.651679,20.00,0.782146,...,17.75,0.711321,21.25,0.766833,21.25,0.704064,19.75,0.712844,16.25,0.714709
ENSG00000142002,DPP9,10.0,18.175,-0.790741,30.25,-0.755341,19.00,-0.817381,18.00,-0.808785,...,18.25,-0.857853,12.25,-0.790998,13.75,-0.782503,14.50,-0.811251,21.25,-0.784845
ENSG00000105672,ETV2,10.0,17.450,0.695006,29.50,0.707315,13.00,0.723869,19.25,0.655049,...,26.75,0.742832,19.25,0.706593,12.25,0.663985,8.25,0.669160,18.00,0.659691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000162624,LHX8,1.0,4.250,-0.661514,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.25,-0.661514,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000100908,EMC9,1.0,3.750,0.275034,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.75,0.275034,NaN,NaN,NaN,NaN
ENSG00000108556,CHRNE,1.0,3.250,-0.641954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.25,-0.641954,NaN,NaN
ENSG00000144214,LYG1,1.0,3.000,0.016361,NaN,NaN,NaN,NaN,NaN,NaN,...,3.00,0.016361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
lifespan_2.metrics_average

Metrics(R2=0.8191444901967483, MAE=5.333966978785094, MSE=222.05742597246913, huber=4.458840087515799)

In [29]:
lifespan_2.validation_metrics["R^2"].mean()

0.8420848215012047

In [ ]:
lifespan_2.plot(max_display=30, plot_size=0.4)

In [26]:
lifespan_2.selected.to_csv(locations.output.stage_two / "lifespan.tsv", sep="\t", index_label="reference_gene")

In [ ]:
summary_stage_two_plot_path = locations.output.dir / "plots" / "stage_two_summary_lifespan.svg"
lifespan_2.plot(title=40, plot_size=0.5, save=summary_stage_two_plot_path )

In [ ]:
lifespan_2.write(partitions_folder , "lifespan_stage_two")